In [ ]:
import pandas as pd
import difflib


movies_data = pd.read_csv('Movies.csv')

def get_closest_movie_titles(movie_name, all_titles, n=5):
    all_titles = [title for title in all_titles if isinstance(title, str)]
    close_matches = difflib.get_close_matches(movie_name, all_titles, n=n)
    return close_matches

print("Welcome to Movie Recommendation Engine!")
print("Enter 'Nothing' if you don't know anything and 'Yes' if any!")



user_input = input("What's on your mind? ").lower()

if user_input == 'nothing':
    print("Showing all Top genres of all categories")
    top_movies = movies_data.head(15)
    print("Top 15 movies across genres:")
    print(top_movies[['index', 'title', 'genre']].to_string(index=False, col_space=[10, 40, 60], header=["Index\t", "Title\t\t\t", "Genre"]))

elif user_input == 'yes':
    filter_technique = input("Enter the filter technique:\n1. Rating\n2. Popularity\n3. Runtime\n4. Genre\n5. Title\n6. Cast\n7. Director\n\nEnter the filter number: ")

    filter_mapping = {
        '1': 'vote_avg',
        '2': 'popularity',
        '3': 'runtime',
        '4': 'genre',
        '5': 'title',
        '6': 'cast',
        '7': 'director'
    }
    if filter_technique in filter_mapping:
        selected_filter = filter_mapping[filter_technique]

        
        
        if selected_filter == 'vote_avg':
            movies_data['vote_avg'] = pd.to_numeric(movies_data['vote_avg'], errors='coerce')
            movies_data.sort_values('vote_avg', ascending=False, inplace=True)
            top_movies = movies_data.head(5)
            print("Top 5 movies based on Rating:")
            print(top_movies[['index', 'title', 'vote_avg']].to_string(index=False, col_space=[10, 40, 15], header=["Index\t", "Title\t\t\t", "Rating"]))

            
            
        elif selected_filter == 'popularity':
            movies_data['popularity'] = pd.to_numeric(movies_data['popularity'], errors='coerce')
            movies_data.sort_values('popularity', ascending=False, inplace=True)
            top_movies = movies_data.head(5)
            print("Top 5 movies based on Popularity:")
            print(top_movies[['index', 'title', 'popularity']].to_string(index=False, col_space=[10, 40, 15], header=["Index\t", "Title\t\t\t", "Popularity"]))

            
            
        elif selected_filter == 'runtime':
            runtime_input = input("Enter a runtime value: ")
            runtime_preference = input("Enter your preference ('greater', 'less', or 'equal'): ")

            try:
                runtime_input = int(runtime_input)
                movies_data['runtime'] = pd.to_numeric(movies_data['runtime'], errors='coerce')

                if runtime_preference == 'greater':
                    filtered_movies = movies_data[movies_data['runtime'] > runtime_input]
                    filtered_movies.sort_values('runtime', ascending=True, inplace=True)
                    top_movies = filtered_movies.head(5)
                    print(f"Top 5 movies with runtime greater than {runtime_input} minutes:")
                    print(top_movies[['index', 'title', 'runtime']].to_string(index=False, col_space=[10, 40, 15], header=["Index\t", "Title\t\t\t", "Runtime"]))

                elif runtime_preference == 'less':
                    filtered_movies = movies_data[movies_data['runtime'] < runtime_input]
                    filtered_movies.sort_values('runtime', ascending=False, inplace=True)
                    top_movies = filtered_movies.head(5)
                    print(f"Top 5 movies with runtime less than {runtime_input} minutes:")
                    print(top_movies[['index', 'title', 'runtime']].to_string(index=False, col_space=[10, 40, 15], header=["Index\t", "Title\t\t\t", "Runtime"]))

                elif runtime_preference == 'equal':
                    filtered_movies = movies_data[movies_data['runtime'] == runtime_input]
                    filtered_movies.sort_values('runtime', ascending=True, inplace=True)
                    top_movies = filtered_movies.head(5)
                    print(f"Top 5 movies with runtime equal to {runtime_input} minutes:")
                    print(top_movies[['index', 'title', 'runtime']].to_string(index=False, col_space=[10, 40, 15], header=["Index\t", "Title\t\t\t", "Runtime"]))

                else:
                    print("Invalid runtime preference! Please enter 'greater', 'less', or 'equal'.")

            except ValueError:
                print("Invalid runtime value! Please enter a valid number.")

                
                
        elif selected_filter == 'genre':
            genre_input = input("Enter a genre: ")
            close_matches = get_closest_movie_titles(genre_input, movies_data['genre'], n=5)

            if close_matches:
                print(f"Top 5 movies with the genre '{genre_input}':")
                matching_movies = movies_data[movies_data['genre'].isin(close_matches)].head(5)
                for index, row in matching_movies.iterrows():
                    print(f"Index: {index}\tTitle: {row['title']}\tGenre: {row['genre']}")
            else:
                print(f"No close matches found for the genre '{genre_input}'.")

                
                
        elif selected_filter == 'title':
            movie_name = input("Enter the movie title: ")
            close_matches = get_closest_movie_titles(movie_name, movies_data['title'], n=5)

            if close_matches:
                print(f"Closest movie titles to '{movie_name}':")
                for match in close_matches:
                    movie_index = movies_data[movies_data['title'] == match].index[0]
                    print(f"Index: {movie_index}\tTitle: {match}")
            else:
                print(f"No close matches found for the movie title '{movie_name}'.")

                
                
        elif selected_filter == 'cast':
            cast_name = input("Enter the cast name: ")
            close_matches = get_closest_movie_titles(cast_name, movies_data['cast'], n=5)

            if close_matches:
                print(f"Closest movies featuring '{cast_name}':")
                for match in close_matches:
                    matching_movies = movies_data[movies_data['cast'].str.split(',').apply(lambda x: cast_name in x)]
                    for index, row in matching_movies.iterrows():
                        print(f"Index: {index}\tTitle: {row['title']}")
            else:
                print(f"No close matches found for the cast name '{cast_name}'.")

                
                
        elif selected_filter == 'director':
            director_name = input("Enter the director name: ")
            close_matches = get_closest_movie_titles(director_name, movies_data['director'], n=5)

            if close_matches:
                print(f"Closest movies directed by '{director_name}':")
                movie_indices = movies_data[movies_data['director'].isin(close_matches)].index
                for index in movie_indices:
                    match = movies_data.loc[index, 'director']
                    movie_title = movies_data.loc[index, 'title']
                    print(f"Index: {index}\tTitle: {movie_title}\tDirector: {match}")
            else:
                print(f"No close matches found for the director name '{director_name}'.")

                
                
        else:
            print("Invalid filter technique number! Please enter a valid number.")
    else:
        print("Invalid filter technique number! Please enter a valid number.")
else:
    print("Invalid input! Please enter 'Unknown' or 'Yes'.")
